In [1]:
from google.colab import files
files.upload()  #upload file kaggle.json di folder

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ezzy19","key":"670b5f2b6ab9d2ec36cba0e69df39cb7"}'}

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permiss

kaggle.json


In [3]:
#mendapatkan dataset dari kaggle
! kaggle datasets download -d yufengdev/bbc-fulltext-and-category

  0% 0.00/1.83M [00:00<?, ?B/s]
100% 1.83M/1.83M [00:00<00:00, 125MB/s]


In [4]:
#unzip file 
import os
!unzip /content/bbc-fulltext-and-category.zip

Archive:  /content/bbc-fulltext-and-category.zip
  inflating: bbc-text.csv            


In [5]:
import pandas as pd
df = pd.read_csv ('/content/bbc-text.csv')
df.head() #melihat 5 data teratas

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [6]:
category = pd.get_dummies(df.category)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns=['category'])
df_baru

,text,business,entertainment,politics,sport,tech
0,tv future in the hands of viewers with home th...,0,0,0,0,1
1,worldcom boss left books alone former worldc...,1,0,0,0,0
2,tigers wary of farrell gamble leicester say ...,0,0,0,1,0
3,yeading face newcastle in fa cup premiership s...,0,0,0,1,0
4,ocean s twelve raids box office ocean s twelve...,0,1,0,0,0
...,...,...,...,...,...,...
2220,cars pull down us retail figures us retail sal...,1,0,0,0,0
2221,kilroy unveils immigration policy ex-chatshow ...,0,0,1,0,0
2222,rem announce new glasgow concert us band rem h...,0,1,0,0,0
2223,how political squabbles snowball it s become c...,0,0,1,0,0


In [11]:
text = df_baru ['text'].values
label = df_baru [['business', 'entertainment', 'politics', 'sport', 'tech' ]].values

In [12]:
from sklearn.model_selection import train_test_split
text_latih, text_test, label_latih, label_test = train_test_split(text, label, test_size = 0.2)

In [ ]:
from tensorflow. keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer (num_words = 5000, oov_token='x')
tokenizer.fit_on_texts(text_latih)
tokenizer.fit_on_texts(text_test)

sekuens_latih =tokenizer.texts_to_sequences(text_latih)
sekuens_test = tokenizer.texts_to_sequences(text_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [ ]:
#mendefinisikan model
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=24),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

#compile model
model.compile(loss='categorical_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])

In [ ]:
class myCallbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy')>0.90):
      print('\nAkurasi telah mencapai >90%!')
      self.model.stop_training = True
callbacks = myCallbacks()

In [ ]:
#training model
history = model.fit(padded_latih, 
                    label_latih, 
                    epochs=30, 
                    validation_data=(padded_test, label_test),
                    verbose=2,
                    callbacks=[callbacks])

Epoch 1/30
56/56 - 22s - loss: 1.6115 - accuracy: 0.2433 - val_loss: 1.6214 - val_accuracy: 0.1551
Epoch 2/30
56/56 - 22s - loss: 1.4809 - accuracy: 0.3084 - val_loss: 2.4573 - val_accuracy: 0.2202
Epoch 3/30
56/56 - 22s - loss: 1.0523 - accuracy: 0.5528 - val_loss: 0.7402 - val_accuracy: 0.6494
Epoch 4/30
56/56 - 22s - loss: 0.7509 - accuracy: 0.6792 - val_loss: 0.7505 - val_accuracy: 0.6831
Epoch 5/30
56/56 - 22s - loss: 0.6090 - accuracy: 0.7343 - val_loss: 0.7867 - val_accuracy: 0.6809
Epoch 6/30
56/56 - 22s - loss: 0.4551 - accuracy: 0.8022 - val_loss: 0.5696 - val_accuracy: 0.7596
Epoch 7/30
56/56 - 22s - loss: 0.2993 - accuracy: 0.8803 - val_loss: 0.5563 - val_accuracy: 0.8000
Epoch 8/30

Akurasi telah mencapai >90%!
56/56 - 22s - loss: 0.1965 - accuracy: 0.9360 - val_loss: 0.5244 - val_accuracy: 0.8404
